In [20]:
from pandas import read_csv
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import statistics
from statsmodels.tsa.stattools import adfuller, kpss, acf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm

In [2]:
plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})

In [115]:
mape_list = []
rmse_list = []
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    mape_list.append(mape)
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    rmse_list.append(rmse)
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 
            'corr':corr})

# Часть 1 - 1й датасет

## 1s timestamp

In [3]:
num = [str(i) for i in range(22)]

In [44]:
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
p2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P2/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
p3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P3/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q1/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q2/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q3/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U1/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U2/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U3/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

In [40]:
def prepare_data(df, num_elements=50):
    new_data = []
    length = df.shape[0]
    current_index = 0
    step = int(0.2 * num_elements)
    while(current_index <= length - num_elements):
        new_data.append(df[current_index:current_index+num_elements])
        current_index += step
    return new_data

In [49]:
p1 = [prepare_data(df) for df in p1]
p1 = sum(p1, [])

p2 = [prepare_data(df) for df in p2]
p2 = sum(p2, [])

p3 = [prepare_data(df) for df in p3]
p3 = sum(p3, [])

q1 = [prepare_data(df) for df in q1]
q1 = sum(q1, [])

q2 = [prepare_data(df) for df in q2]
q2 = sum(q2, [])

q3 = [prepare_data(df) for df in q3]
q3 = sum(q3, [])

u1 = [prepare_data(df) for df in u1]
u1 = sum(u1, [])

u2 = [prepare_data(df) for df in u2]
u2 = sum(u2, [])

u3 = [prepare_data(df) for df in u3]
u3 = sum(u3, [])

In [122]:
def train_arima(df, index=0):
    train, test = train_test_split(df, test_size=0.2, shuffle=False)
    model = pm.auto_arima(train.values, start_p=0, start_q=0,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=True,   # Seasonality
                      start_P=0, 
                      D=1, 
                      trace=False,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)
#     print("Data " + str(index))
#     print(forecast_accuracy(model.predict(n_periods=test.shape[0]), test.values))
    forecast_accuracy(model.predict(n_periods=test.shape[0]), test.values)
#     print("\n")

In [70]:
for df in p1:
    train_arima(df)

Data 0
{'mape': 0.0054215790508355555, 'me': -1.576603366405942, 'mae': 2.1125146801354218, 'mpe': -0.004031388999785659, 'rmse': 2.5750234738609628, 'acf1': 0.14192518151847022, 'corr': 0.2846533377323913, 'minmax': 0.005419167761783528}


Data 0
{'mape': 0.006427297681500463, 'me': -1.806430940106179, 'mae': 2.524412425916876, 'mpe': -0.004575234734798185, 'rmse': 3.3797235366847795, 'acf1': 0.14192518151847022, 'corr': -0.28257547357894564, 'minmax': 0.006423702118887542}




/Users/anna/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:572: FutureWarning: fft=True will become the default in a future version of statsmodels. To suppress this warning, explicitly set fft=False.
  FutureWarning
/Users/anna/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:572: FutureWarning: fft=True will become the default in a future version of statsmodels. To suppress this warning, explicitly set fft=False.
  FutureWarning


Data 0
{'mape': 0.014519716008397698, 'me': -5.3471475426757875, 'mae': 5.8934534936679315, 'mpe': -0.013117802821225367, 'rmse': 9.625988250805618, 'acf1': 0.14192518151847022, 'corr': -0.37755651331042006, 'minmax': 0.01451799731351655}


Data 0
{'mape': 0.009113615525220433, 'me': 2.7929491391612373, 'mae': 3.5691204315351115, 'mpe': 0.007166420792589857, 'rmse': 4.183215819968257, 'acf1': 0.14192518151847022, 'corr': -0.17100053190348619, 'minmax': 0.009010107147190638}




/Users/anna/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:572: FutureWarning: fft=True will become the default in a future version of statsmodels. To suppress this warning, explicitly set fft=False.
  FutureWarning
/Users/anna/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:572: FutureWarning: fft=True will become the default in a future version of statsmodels. To suppress this warning, explicitly set fft=False.
  FutureWarning


Data 0
{'mape': 0.012980108102706101, 'me': 5.052183559275642, 'mae': 5.052183559275642, 'mpe': 0.012980108102706101, 'rmse': 5.356398220197668, 'acf1': 0.14192518151847022, 'corr': -0.2900840876932664, 'minmax': 0.012793398206707662}


Data 0
{'mape': 0.0023971636126114207, 'me': 0.8201607009144652, 'mae': 0.9315138823520386, 'mpe': 0.0021117729689799384, 'rmse': 1.2168897409580366, 'acf1': 0.14192518151847022, 'corr': -0.7983879479766645, 'minmax': 0.0023875251006796328}




...

/Users/anna/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:572: FutureWarning: fft=True will become the default in a future version of statsmodels. To suppress this warning, explicitly set fft=False.
  FutureWarning


Data 0
{'mape': 0.011371116876821151, 'me': 7.979960818470113, 'mae': 8.00707322647113, 'mpe': 0.011332926621442429, 'rmse': 8.961202631510186, 'acf1': 0.14192518151847022, 'corr': -0.20305116235955167, 'minmax': 0.011211453711991504}




/Users/anna/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:572: FutureWarning: fft=True will become the default in a future version of statsmodels. To suppress this warning, explicitly set fft=False.
  FutureWarning


Data 0
{'mape': 0.007118777886173223, 'me': 4.951613260032252, 'mae': 5.0148048954274485, 'mpe': 0.007030222724005862, 'rmse': 5.590184705045857, 'acf1': 0.14192518151847022, 'corr': 0.7722999702642613, 'minmax': 0.0070563151384256395}


Data 0
{'mape': 0.003535206005519081, 'me': -1.1770834268668977, 'mae': 2.5244304155410875, 'mpe': -0.0016303767075004077, 'rmse': 3.3262513134676968, 'acf1': 0.14192518151847022, 'corr': 0.7545184971189737, 'minmax': 0.003531015640135693}




/Users/anna/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:572: FutureWarning: fft=True will become the default in a future version of statsmodels. To suppress this warning, explicitly set fft=False.
  FutureWarning
/Users/anna/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:572: FutureWarning: fft=True will become the default in a future version of statsmodels. To suppress this warning, explicitly set fft=False.
  FutureWarning


In [72]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.36394013118524343
707.6627387259026


In [117]:
mape_list = []
rmse_list = []

In [75]:
# train_multi = pd.concat([p1[0], p2[0], p3[0], q1[0], q2[0], q3[0], u1[0]], axis=1)

In [83]:
# train, test = train_test_split(train_multi, test_size=0.2, shuffle=False)

In [98]:
# from statsmodels.tsa.statespace.varmax import VARMAX

# model = VARMAX(train, order = (1,1))
# model_fit = model.fit()
# print(forecast_accuracy(model_fit.forecast(steps=test.shape[0]), test.values))

In [99]:
# model_fit.forecast(steps=test.shape[0])

In [100]:
# model = ARIMA(train, order=(0, 1, 0))  
# fitted = model.fit(disp=-1)  

# # Forecast
# fc, se, conf = fitted.forecast(len(test), alpha=0.05)  # 95% conf

# # Make as pandas series
# fc_series = pd.Series(fc, index=test.index)
# lower_series = pd.Series(conf[:, 0], index=test.index)
# upper_series = pd.Series(conf[:, 1], index=test.index)
# print(fitted.summary())

# # Plot
# plt.figure(figsize=(12,5), dpi=100)
# plt.plot(train, label='training')
# plt.plot(test, label='actual')
# plt.plot(fc_series, label='forecast')
# plt.fill_between(lower_series.index, lower_series, upper_series, 
#                  color='k', alpha=.15)
# plt.title('Forecast vs Actuals')
# plt.legend(loc='upper left', fontsize=8)
# plt.show()

## 1.5s timestamp

In [107]:
num = [str(i) for i in range(141)]

In [108]:
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
p2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P2/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
p3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P3/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q1/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q2/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q3/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U1/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U2/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U3/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

In [109]:
p1 = [prepare_data(df) for df in p1]
p1 = sum(p1, [])

p2 = [prepare_data(df) for df in p2]
p2 = sum(p2, [])

p3 = [prepare_data(df) for df in p3]
p3 = sum(p3, [])

q1 = [prepare_data(df) for df in q1]
q1 = sum(q1, [])

q2 = [prepare_data(df) for df in q2]
q2 = sum(q2, [])

q3 = [prepare_data(df) for df in q3]
q3 = sum(q3, [])

u1 = [prepare_data(df) for df in u1]
u1 = sum(u1, [])

u2 = [prepare_data(df) for df in u2]
u2 = sum(u2, [])

u3 = [prepare_data(df) for df in u3]
u3 = sum(u3, [])

In [118]:
for df in p1:
    train_arima(df)

Data 0
{'mape': 0.0024802125467275676, 'me': -1.1202994057226987, 'mae': 2.010067403443986, 'mpe': -0.001376399142732841, 'rmse': 2.6084504050181905, 'corr': 0.18256665153260412}


Data 0
{'mape': 0.0015257328604208887, 'me': 0.768030379466154, 'mae': 1.2319961823413905, 'mpe': 0.0009530700275335368, 'rmse': 1.5664102966781248, 'corr': 0.23398927436580913}


Data 0
{'mape': 0.0017454424034204865, 'me': 1.2907382200138044, 'mae': 1.4094404194347363, 'mpe': 0.0015989181855010034, 'rmse': 1.6280624984643481, 'corr': 0.4120757587197652}


Data 0
{'mape': 0.002324391401906217, 'me': -0.8790559730536301, 'mae': 1.8813924038081382, 'mpe': -0.0010809276320258754, 'rmse': 2.153567911141845, 'corr': -0.6208386507179126}


Data 0
{'mape': 0.0015465795899011108, 'me': 0.05873660573148527, 'mae': 1.2518463922001843, 'mpe': 7.74625402362653e-05, 'rmse': 1.7846237010188362, 'corr': -0.5126342709765163}


Data 0
{'mape': 0.0021562923082058083, 'me': 1.7375132797560924, 'mae': 1.7375132797560924, 'mpe'

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


Data 0
{'mape': 0.0023623752309801078, 'me': 1.9153046448507554, 'mae': 1.9153046448507554, 'mpe': 0.0023623752309801078, 'rmse': 2.345139164363381, 'corr': -0.42148677137383783}


Data 0
{'mape': 0.005300241469076477, 'me': -3.826016662327288, 'mae': 4.34806062737407, 'mpe': -0.0046562549647960316, 'rmse': 5.988711838796357, 'corr': 0.01543309878829571}


Data 0
{'mape': 0.004143858192783653, 'me': 2.8656066186735414, 'mae': 3.3673573672434713, 'mpe': 0.0035311712901968427, 'rmse': 3.622607955811159, 'corr': 0.3424977191207999}


Data 0
{'mape': 0.002709047742423638, 'me': 2.193595532280426, 'mae': 2.193595532280426, 'mpe': 0.002709047742423638, 'rmse': 3.0721200257400154, 'corr': -0.04177863742934886}


Data 0
{'mape': 0.0019749470766445036, 'me': 0.03423139600967033, 'mae': 1.6056160937121944, 'mpe': 4.723248680203896e-05, 'rmse': 1.8545455149003298, 'corr': -0.5071141648111901}


Data 0
{'mape': 0.0022094790748045496, 'me': -0.6083380104884213, 'mae': 1.7953935384009696, 'mpe': -0.

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


Data 0
{'mape': 0.005789071087147677, 'me': 4.675264351724627, 'mae': 4.675264351724627, 'mpe': 0.005789071087147677, 'rmse': 4.977479573633694, 'corr': 0.4073933650452088}


Data 0
{'mape': 0.00946655631898404, 'me': 7.6881624059415685, 'mae': 7.6881624059415685, 'mpe': 0.00946655631898404, 'rmse': 8.585986658983945, 'corr': 0.7083055017876937}


Data 0
{'mape': 0.010456184892636947, 'me': 6.358824582663499, 'mae': 8.49618607001313, 'mpe': 0.007875758472999993, 'rmse': 9.028034747987528, 'corr': 0.6468734316182531}


Data 0
{'mape': 0.0050482949931072284, 'me': 3.4615000000000125, 'mae': 4.1065000000000165, 'mpe': 0.004262543290645201, 'rmse': 4.393055599466034, 'corr': -3.78257455194286e-14}


Data 0
{'mape': 0.004112050088719236, 'me': 3.337389539174603, 'mae': 3.337389539174603, 'mpe': 0.004112050088719236, 'rmse': 3.613034488174614, 'corr': 0.14245156474307505}


Data 0
{'mape': 0.003976401068514145, 'me': -2.746166466615546, 'mae': 3.2347857298452025, 'mpe': -0.003372555454187079

...

Data 0
{'mape': 0.05392747822767259, 'me': 35.039329511781695, 'mae': 35.039329511781695, 'mpe': 0.05392747822767259, 'rmse': 37.29304169188457, 'corr': 0.770269544119877}


Data 0
{'mape': 0.02786097861612428, 'me': 18.024435044545875, 'mae': 18.024435044545875, 'mpe': 0.02786097861612428, 'rmse': 18.577381791783488, 'corr': -0.7073479301475625}


Data 0
{'mape': 0.003318762755557759, 'me': 0.03551654610158721, 'mae': 2.16046673768941, 'mpe': 7.969358566115495e-05, 'rmse': 3.235660790671509, 'corr': 0.2205060207632188}


Data 0
{'mape': 0.004615713594908261, 'me': -1.2605420125985574, 'mae': 3.005048607924232, 'mpe': -0.0019190683736722072, 'rmse': 3.3455801383110626, 'corr': -0.28516418594415455}


Data 0
{'mape': 0.005474223351609475, 'me': 2.8445330501014383, 'mae': 3.549868518507117, 'mpe': 0.004397547905069335, 'rmse': 3.6391252874550912, 'corr': 0.14747893514830995}


Data 0
{'mape': 0.002542078593820986, 'me': -0.46781386434365685, 'mae': 1.654004120206946, 'mpe': -0.0007099314

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


Data 0
{'mape': 0.003062194576135299, 'me': 1.9862158043662248, 'mae': 1.9862158043662248, 'mpe': 0.003062194576135299, 'rmse': 2.2164797448696785, 'corr': 0.27724474157448287}


Data 0
{'mape': 0.0036724912946612524, 'me': -2.225709454931041, 'mae': 2.419790305593563, 'mpe': -0.0033733047122245243, 'rmse': 4.8487907781943935, 'corr': 0.10263710376214125}


Data 0
{'mape': 0.000992050415087166, 'me': -0.45651965628152313, 'mae': 0.6446455476424149, 'mpe': -0.0007021380778835556, 'rmse': 0.8953393226922641, 'corr': -0.07956317794411137}


Data 0
{'mape': 0.0014034643894172024, 'me': 0.857246288243448, 'mae': 0.9109397510243411, 'mpe': 0.0013209339742849481, 'rmse': 1.021799133546141, 'corr': 0.7083298829150053}


Data 0
{'mape': 0.00166323053051461, 'me': -0.7518108110880007, 'mae': 1.0847000404007416, 'mpe': -0.001149903394155403, 'rmse': 1.7899872106841133, 'corr': 0.05676780479212978}


Data 0
{'mape': 0.004866497856239623, 'me': -2.4026250000000346, 'mae': 3.1895249999999806, 'mpe':

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


Data 0
{'mape': 0.0030785718376279666, 'me': -1.891306504152692, 'mae': 2.0145388133646636, 'mpe': -0.0028887733537630864, 'rmse': 3.352729225118693, 'corr': -0.22429598531255968}


Data 0
{'mape': 0.002679321711699666, 'me': 0.02351597043328866, 'mae': 1.748575163750445, 'mpe': 4.660831520723336e-05, 'rmse': 2.0971990930586646, 'corr': 0.34044405082839946}


Data 0
{'mape': 0.5723877358851279, 'me': -952.8885539041485, 'mae': 952.8885539041485, 'mpe': -0.5723877358851279, 'rmse': 989.3737252178661, 'corr': 0.5174446037575119}


Data 0
{'mape': 0.25714957395095217, 'me': 434.9016017727489, 'mae': 434.9016017727489, 'mpe': 0.25714957395095217, 'rmse': 494.67804876946474, 'corr': 0.5851788631048922}


Data 0
{'mape': 0.09667230832344992, 'me': 163.1680901057585, 'mae': 163.1680901057585, 'mpe': 0.09667230832344992, 'rmse': 184.35674329408263, 'corr': -0.20834233772046287}


Data 0
{'mape': 0.05955364383579535, 'me': -100.1811412416306, 'mae': 100.1811412416306, 'mpe': -0.0595536438357953

In [119]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.1436254575969297
284.48231966576367


## 2s timestamp

In [120]:
num = [str(i) for i in range(902)]

In [123]:
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
p2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P2/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
p3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P3/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q1/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q2/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q3/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U1/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U2/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U3/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

In [124]:
p1 = [prepare_data(df) for df in p1]
p1 = sum(p1, [])

p2 = [prepare_data(df) for df in p2]
p2 = sum(p2, [])

p3 = [prepare_data(df) for df in p3]
p3 = sum(p3, [])

q1 = [prepare_data(df) for df in q1]
q1 = sum(q1, [])

q2 = [prepare_data(df) for df in q2]
q2 = sum(q2, [])

q3 = [prepare_data(df) for df in q3]
q3 = sum(q3, [])

u1 = [prepare_data(df) for df in u1]
u1 = sum(u1, [])

u2 = [prepare_data(df) for df in u2]
u2 = sum(u2, [])

u3 = [prepare_data(df) for df in u3]
u3 = sum(u3, [])

In [125]:
for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [126]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.12204952696294605
80.56897564126488


## 5s timestamp

In [128]:
num = [str(i) for i in range(19)]

In [129]:
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
p2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P2/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
p3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P3/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q1/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q2/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q3/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U1/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U2/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U3/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

In [130]:
p1 = [prepare_data(df) for df in p1]
p1 = sum(p1, [])

p2 = [prepare_data(df) for df in p2]
p2 = sum(p2, [])

p3 = [prepare_data(df) for df in p3]
p3 = sum(p3, [])

q1 = [prepare_data(df) for df in q1]
q1 = sum(q1, [])

q2 = [prepare_data(df) for df in q2]
q2 = sum(q2, [])

q3 = [prepare_data(df) for df in q3]
q3 = sum(q3, [])

u1 = [prepare_data(df) for df in u1]
u1 = sum(u1, [])

u2 = [prepare_data(df) for df in u2]
u2 = sum(u2, [])

u3 = [prepare_data(df) for df in u3]
u3 = sum(u3, [])

In [131]:
for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [132]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.36568570688879726
379.52280233551335


## 15s timestamp

In [133]:
num = [str(i) for i in range(20)]

In [134]:
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
p2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P2/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
p3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P3/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q1/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q2/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
q3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/Q3/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U1/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u2 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U2/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]
u3 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/U3/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

In [135]:
p1 = [prepare_data(df) for df in p1]
p1 = sum(p1, [])

p2 = [prepare_data(df) for df in p2]
p2 = sum(p2, [])

p3 = [prepare_data(df) for df in p3]
p3 = sum(p3, [])

q1 = [prepare_data(df) for df in q1]
q1 = sum(q1, [])

q2 = [prepare_data(df) for df in q2]
q2 = sum(q2, [])

q3 = [prepare_data(df) for df in q3]
q3 = sum(q3, [])

u1 = [prepare_data(df) for df in u1]
u1 = sum(u1, [])

u2 = [prepare_data(df) for df in u2]
u2 = sum(u2, [])

u3 = [prepare_data(df) for df in u3]
u3 = sum(u3, [])

In [136]:
for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [137]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.2987975368463387
248.6100359520287


### length 100

In [140]:
num = [str(i) for i in range(22)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

In [141]:
p1 = [prepare_data(df, 100) for df in p1]
p1 = sum(p1, [])

In [142]:
for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [143]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

4.027712089885608
11133.423183409586


In [144]:
num = [str(i) for i in range(141)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 100) for df in p1]
p1 = sum(p1, [])

In [145]:
for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [146]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.6442096230689787
1365.9769325486047


In [147]:
num = [str(i) for i in range(902)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 100) for df in p1]
p1 = sum(p1, [])

In [148]:
for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [149]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.2311062676320719
197.68291687390177


In [150]:
num = [str(i) for i in range(19)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 100) for df in p1]
p1 = sum(p1, [])

In [151]:
for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [152]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.19560106118987197
208.79425118864012


In [153]:
num = [str(i) for i in range(20)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 100) for df in p1]
p1 = sum(p1, [])

In [154]:
for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [155]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.2491311137419793
216.13297039291757


### length 150

In [157]:
def print_and_clear_error():
    print(sum(mape_list) / len(mape_list))
    print(sum(rmse_list) / len(rmse_list))
    mape_list = []
    rmse_list = []

In [158]:
num = [str(i) for i in range(22)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 150) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [161]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

22.93535527865276
65344.43665744977


In [162]:
num = [str(i) for i in range(141)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 150) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [163]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

1.4472671001739477
2775.323559395702


In [164]:
num = [str(i) for i in range(902)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 150) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [165]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.19592970874823062
78.99281548263248


In [166]:
num = [str(i) for i in range(19)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 150) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [167]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.2196171180947174
253.91716619579242


In [168]:
num = [str(i) for i in range(20)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 150) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [ ]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.5003661250567334
327.26195839882223


### length 200

In [170]:
num = [str(i) for i in range(22)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 200) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [171]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.8039022797380683
167.3453519292546


In [172]:
num = [str(i) for i in range(141)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 200) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [173]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.17757639463394395
334.4779965746515


In [174]:
num = [str(i) for i in range(902)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 200) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [175]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

1.9693278906565406
4243.576525592701


In [176]:
num = [str(i) for i in range(19)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 200) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [177]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.21505807034262073
267.16761413164414


In [178]:
num = [str(i) for i in range(20)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 200) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [179]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.272212130690957
266.72408343429856


### length 250

In [180]:
num = [str(i) for i in range(22)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 250) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [181]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.3445859821448919
158.9594700118599


In [182]:
num = [str(i) for i in range(141)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/1500/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 250) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [183]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.1871466464152989
319.3738586696465


In [184]:
num = [str(i) for i in range(902)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/2000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 250) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [185]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.7669413188813033
242.18205288305424


In [186]:
num = [str(i) for i in range(19)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/5000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 250) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [187]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.25793478665752584
307.53382284605533


In [188]:
num = [str(i) for i in range(20)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/15000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 250) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

In [189]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

0.21018388599787513
210.51874273013163


In [192]:
num = [str(i) for i in range(22)]
p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/1000/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

p1 = [prepare_data(df, 300) for df in p1]
p1 = sum(p1, [])

for df in p1:
    train_arima(df)

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [193]:
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

789.7486518049775
2274389.7738059782


### length 300

In [238]:
def learn(all_num, sample_size, ts):
    num = [str(i) for i in range(all_num)]
    p1 = [read_csv('ics_data_preprocessed/06_Smart_Meter_45/P1/' + str(ts) + '/' + n + '.csv', squeeze=True, parse_dates=['ts'], index_col='ts') for n in num]

    p1 = [prepare_data(df, sample_size) for df in p1]
    p1 = sum(p1, [])
    
    if not p1:
        print("Not enough data")
        return
    else:
        print("Data count: ", len(p1))

    for df in p1:
        train_arima(df)

In [217]:
learn(22, 300, 1000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


789.7486518049775
2274389.7738059782


In [219]:
learn(141, 300, 1500)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.2248273553307711
380.0225047656482


In [222]:
learn(902, 300, 2000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.2636430514722493
122.84868265562429


In [223]:
learn(19, 300, 5000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.26791404431479415
331.78760299837006


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [224]:
learn(20, 300, 15000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.2105469337522933
236.4513355492472


### length 350

In [226]:
learn(22, 350, 1000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

2.087758299003998
402.8233690586467


In [227]:
learn(141, 350, 1500)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.21305206482188824
393.7276960486906


In [228]:
learn(902, 350, 2000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

138.13395234137604
132059.33167803346


In [229]:
learn(19, 350, 5000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.2895345013421249
340.5400714462001


In [230]:
learn(20, 350, 15000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.39005526695654613
349.5941508291416


### length 400

In [231]:
learn(22, 400, 1000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

2.6851620268940457
456.67507795870006


In [232]:
learn(141, 400, 1500)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.21035215023488904
377.341264379525


In [233]:
learn(902, 400, 2000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

27.006705968968305
31944.056306466682


In [234]:
learn(19, 400, 5000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.28998716046216844
343.69498820339606


In [235]:
learn(20, 400, 15000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.210395815675285
208.86795105794522


### length 450

In [236]:
learn(22, 450, 1000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

4911.982051910316
13843509.93535167


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [237]:
learn(141, 450, 1500)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.26021152636469935
494.2777986923073


In [239]:
learn(902, 450, 2000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  1163


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.2758831728373758
89.96928072374111


In [240]:
learn(19, 450, 5000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  262


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.36342419911416796
337.82606171211586


In [241]:
learn(20, 450, 15000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  143


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.2075128164280248
208.9427548760129


### length 500

In [242]:
learn(22, 500, 1000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  6
2.520392625736497
625.0468799479837


In [243]:
learn(141, 500, 1500)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  20


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.2529827110259668
573.6826603758407


In [244]:
learn(902, 500, 2000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  999


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.20232879863159534
78.64835748353332


In [245]:
learn(19, 500, 5000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  224


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.27671854713631144
326.75171109072954


In [246]:
learn(20, 500, 15000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  122


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.27188671231896117
247.89873386360824


### length 550

In [247]:
learn(22, 550, 1000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  5
0.9929728218762195
384.1197016191743


In [248]:
learn(141, 550, 1500)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  11
0.2647541841968913
551.3217557723925


In [249]:
learn(902, 550, 2000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  867


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.39556870509375563
122.23825097472495


In [252]:
learn(19, 550, 5000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  202


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.27819003909103285
344.13105693582025


In [253]:
learn(20, 550, 15000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  105


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.2537443030525345
240.86823612727792


### length 600

In [254]:
learn(22, 600, 1000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  4
4.24519401993838
961.5147013338412


In [255]:
learn(141, 600, 1500)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  6
0.45567856456677586
759.9314575794983


In [256]:
learn(902, 600, 2000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  747


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.2320983940629905
83.06171106705555


In [257]:
learn(19, 600, 5000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  176


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.29533608499916847
321.0909254876215


In [258]:
learn(20, 600, 15000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  90


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.27846888063133096
256.26738707449215


### length 650

In [259]:
learn(22, 650, 1000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  3
0.3605654085505419
690.8345890778609


In [260]:
learn(141, 650, 1500)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  5
0.1683615719348471
383.0769657218505


In [261]:
learn(902, 650, 2000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  652


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.4800098806521107
146.2921058164135


In [262]:
learn(19, 650, 5000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  155


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.32137889587454577
308.37643853250006


In [263]:
learn(20, 650, 15000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  77


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.31548292471849393
256.66751658418184


### length 700

In [264]:
learn(22, 700, 1000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  3
3.647264081619951
691.9679299673479


In [266]:
learn(141, 700, 1500)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  3
0.19888337058563052
671.6705051647799


In [267]:
learn(902, 700, 2000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  579


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

83.37985952676641
20692.467486350433


In [268]:
learn(19, 700, 5000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  144


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.30616055716252183
312.68152853387323


In [269]:
learn(20, 700, 15000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  67


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.34386377238202437
257.3023257311414


### length 750

In [270]:
learn(22, 750, 1000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  2
8.719171035518833
1126.943662489378


In [271]:
learn(141, 750, 1500)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  2
0.19803593700030192
413.6425114714287


In [272]:
learn(902, 750, 2000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  507


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.5209265769502287
184.32657308610823


In [273]:
learn(19, 750, 5000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  127


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.36544683713969545
321.5026299733146


In [274]:
learn(20, 750, 15000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  59
0.3615161208554485
291.8700818030408


### length 800

In [275]:
learn(22, 800, 1000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  2
10.417213728524999
1235.44788972255


In [276]:
learn(141, 800, 1500)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  2
0.27464937223929553
486.2802432817851


In [277]:
learn(902, 800, 2000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  456


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.3599585661730351
124.73048699086056


In [278]:
learn(19, 800, 5000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  115


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.379098214924801
337.3252474676892


In [279]:
learn(20, 800, 15000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  51


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.34747043013195983
217.18657406965426


### length 850

In [280]:
learn(22, 850, 1000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  1
0.04893720957029707
6.692814946424489


In [282]:
learn(902, 850, 2000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  411


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.33320911215914367
120.37135480715727


In [283]:
learn(19, 850, 5000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  104


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.372213699667597
334.04090353752383


In [284]:
learn(20, 850, 15000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  44


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.3100415412290853
229.7350840748356


### length 900

In [285]:
learn(22, 900, 1000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  1
0.2014111814475579
237.42240613808076


In [286]:
learn(902, 900, 2000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  362


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.2411611327715149
100.11122922121956


In [287]:
learn(19, 900, 5000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  97


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.44720012620594174
345.99637180265273


In [288]:
learn(20, 900, 15000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  39


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.3521256608172912
201.24576067594793


### length 950

In [289]:
learn(22, 950, 1000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  1
0.39572081478233306
378.43393924732993


In [290]:
learn(902, 950, 2000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  333


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.3115873425277766
115.89556242740673


In [291]:
learn(19, 950, 5000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  88


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.41476539145274427
329.304276723243


In [292]:
learn(20, 950, 15000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  32


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.27798393003213556
207.35556261538463


### length 1000

In [293]:
learn(22, 1000, 1000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  1
0.5849554206395091
478.63721780104214


In [294]:
learn(902, 1000, 2000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  298


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/

0.2633112752047433
105.94667256326537


In [296]:
learn(19, 1000, 5000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  80


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.4482950145740592
324.14632155851666


In [295]:
learn(20, 1000, 15000)
print(sum(mape_list) / len(mape_list))
print(sum(rmse_list) / len(rmse_list))
mape_list = []
rmse_list = []

Data count:  30


/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/anna/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


0.34953870068502474
222.2684048897492
